In [1]:
# class + ORM

In [2]:
# 순서
# declarative_base

# sqlalchemy 구조 복습하기***
# 새로운 개념(까먹은 개념): Meta Data, Session, commit

In [53]:
from sqlalchemy.engine import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm.session import sessionmaker

In [2]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
base = declarative_base()

In [4]:
class User(base):
    __tablename__ = 'T_USER'
    __table_args__ = {'extend_existing':True} # 테이블 객체에 있었다.

    pk = Column('PK',  Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)

    def __repr__(self):
        return f'pk={self.pk}. name={self.name}'

# base에 등록된 User 클래스 - 아래의 테이블과 다르다.

In [5]:
# ORM - tabel 객체
base.metadata.tables 

FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_USER>, nullable=False), schema=None)})

In [6]:
base.metadata.create_all(engine) # 이때 테이블이 생성된다.
                                 # Table 객체를 DB에 반영(테이블 생성시)

2024-03-18 11:21:59,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 11:21:59,011 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2024-03-18 11:21:59,012 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 11:21:59,014 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2024-03-18 11:21:59,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 11:21:59,016 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2024-03-18 11:21:59,017 INFO sqlalchemy.engine.Engine [no key 0.00082s] ()
2024-03-18 11:21:59,018 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
Mapping; base      ORM;core    DB
declarative_base   Metadata    RDBMS
Class(base)        Table객체    Table 생김

SyntaxError: invalid syntax (1939222039.py, line 1)

In [7]:
a = User(name='kim')

In [8]:
a # 아직 DB에 안들어가서 PK 값이 없다.

pk=None. name=kim

In [9]:
# 세션만들기
session = sessionmaker(engine)
sess = session()

In [10]:
# session 객체가 주시하고 있는 클래스의 인스턴스를 등록하는 과정
sess.add(a)

In [11]:
sess.dirty # 세션이 관측하고 있는 객체에 변화가 감지 되었는가.

IdentitySet([])

In [12]:
sess.is_modified(a) # 특정 객체가 어떻게 되었는가 변했는가?

True

In [13]:
sess.commit()

2024-03-18 11:22:16,225 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 11:22:16,228 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2024-03-18 11:22:16,229 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('kim',)
2024-03-18 11:22:16,230 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
sess.is_modified(a)

False

In [15]:
a

2024-03-18 11:22:18,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 11:22:18,507 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2024-03-18 11:22:18,508 INFO sqlalchemy.engine.Engine [generated in 0.00138s] (1,)


pk=1. name=kim

In [16]:
a.name = '이름 바뀜'

In [17]:
a

pk=1. name=이름 바뀜

In [18]:
sess.dirty

IdentitySet([pk=1. name=이름 바뀜])

In [19]:
sess.commit()

2024-03-18 11:22:22,991 INFO sqlalchemy.engine.Engine UPDATE "T_USER" SET "NAME"=? WHERE "T_USER"."PK" = ?
2024-03-18 11:22:22,995 INFO sqlalchemy.engine.Engine [generated in 0.00365s] ('이름 바뀜', 1)
2024-03-18 11:22:22,997 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
sess.query(User).all() is a
sess.query(User).filter(User.pk == 1).all()[0] is a
sess.query(User).all()[0]

2024-03-18 11:22:24,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 11:22:24,602 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2024-03-18 11:22:24,603 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ()
2024-03-18 11:22:24,605 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2024-03-18 11:22:24,606 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1,)
2024-03-18 11:22:24,606 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2024-03-18 11:22:24,607 INFO sqlalchemy.engine.Engine [cached since 0.00541s ago] ()


pk=1. name=이름 바뀜

In [21]:
# 메타데이터를 만들어서 싱크 시킬 수도 있고
# reflect를 통해 데이터베이스에 직접 소통가능

In [22]:
base.registry.dispose()

In [23]:
base.metadata.tables

FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_USER>, nullable=False), schema=None)})

In [24]:
# 메타데이터를 따로 관리하는 경우
class User(base):
    __table__ = base.metadata.tables['T_USER']
    

In [25]:
# sess.query(User).all() is a
# sess.query(User).filter(User.pk == 1).all()[0] is a
sess.query(User).all()[0]

2024-03-18 11:22:30,785 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2024-03-18 11:22:30,786 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ()


In [26]:
from sqlalchemy.schema import Table
class User(base):
    __table__ = Table('T_USER', base.metadata, reflect=True)
    __table_args__ = { 'extend_existing':True }

/var/folders/1n/x_wfn1w17ng2dw0lf7_z5klw0000gn/T/ipykernel_70757/771683204.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  class User(base):


In [27]:
class Address(base):
    __tablename__ = 'T_ADDRESS'
    __table_args__ = { 'extend_existing':True }

    pk = Column('PK', Integer, primary_key = True)
    address = Column('ADDRESS', Text)
    fk = ForeignKey(User.PK)

In [28]:
base.metadata.create_all(engine)

2024-03-18 11:30:24,638 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 11:30:24,640 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2024-03-18 11:30:24,642 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 11:30:24,643 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2024-03-18 11:30:24,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 11:30:24,644 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ADDRESS")
2024-03-18 11:30:24,645 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 11:30:24,646 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ADDRESS" (
	"PK" INTEGER NOT NULL, 
	"ADDRESS" TEXT, 
	PRIMARY KEY ("PK")
)


2024-03-18 11:30:24,647 INFO sqlalchemy.engine.Engine [no key 0.00058s] ()
2024-03-18 11:30:24,648 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
a = sess.query(User).one()

2024-03-18 11:30:53,021 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2024-03-18 11:30:53,022 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()


In [33]:
sess.add(Address(address='주소', fk=a.PK))

In [36]:
sess.dirty, sess.is_modified(a) 

(IdentitySet([]), False)

In [37]:
sess.commit()

2024-03-18 11:32:13,333 INFO sqlalchemy.engine.Engine INSERT INTO "T_ADDRESS" ("ADDRESS") VALUES (?)
2024-03-18 11:32:13,339 INFO sqlalchemy.engine.Engine [generated in 0.00623s] ('주소',)
2024-03-18 11:32:13,342 INFO sqlalchemy.engine.Engine COMMIT


AttributeError: 'NoneType' object has no attribute 'expire'

In [113]:
sess.close()

2024-03-18 12:13:48,257 INFO sqlalchemy.engine.Engine ROLLBACK


In [114]:
engine.dispose() 

In [115]:
### 다시

In [116]:
base = declarative_base()
engine = create_engine('sqlite:///:memory:')

In [42]:
class User(base):
    __tablename__ = 'T_USER'

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)

class Address(base):
    __tablename__ = 'T_ADDRESS'

    pk = Column('PK', Integer, primary_key=True)
    address = Column('NAME',Text, nullable=False)
    fk = Column('FK', Integer, nullable=False)

In [43]:
base.metadata.create_all(engine)

In [117]:
session = sessionmaker(engine)
sess = session()

In [46]:
sess.add(User(name='이름1')) 

In [70]:
a = sess.query(User).one()
print(a.pk)

1


In [48]:
sess.add(Address(address='주소1',fk=a.pk))

In [49]:
sess.query(Address).all()[0].fk

1

In [50]:
from sqlalchemy.sql import join
sess.query(User.name, Address.address).select_from(join(User, Address, User.pk==Address.fk)).all()

[('이름1', '주소1')]

In [ ]:

class Phone(base):
    __tablename__ = 'T_PHONE'

    pk = Column('PK', Integer, primary_key=True)
    phone = Column('PHONE', Text)
    fk = ForeignKey(User.pk)

In [63]:
base.metadata.create_all(engine)

In [74]:
sess.add(Phone(phone='전화번호', fk=a.pk ))

In [75]:
sess.commit() 

In [79]:
sess.close_all()
base.metadata.clear()
engine.dispose()

/var/folders/1n/x_wfn1w17ng2dw0lf7_z5klw0000gn/T/ipykernel_70757/2840527228.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  sess.close_all()


In [80]:
## 다시

In [118]:
sess.close()
engine.dispose()
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [106]:
base = declarative_base()

class User(base):
    __tablename__ = 'T_USER'

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)

class Address(base):
    __tablename__ = 'T_ADDRESS'

    pk = Column('PK', Integer, primary_key=True)
    address = Column('NAME',Text, nullable=False)
    fk = Column('FK', Integer, ForeignKey(User.name))

In [107]:
base.metadata.create_all(engine)

2024-03-18 12:13:09,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 12:13:09,026 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2024-03-18 12:13:09,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 12:13:09,030 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2024-03-18 12:13:09,030 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 12:13:09,041 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2024-03-18 12:13:09,041 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 12:13:09,042 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ADDRESS")
2024-03-18 12:13:09,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 12:13:09,043 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2024-03-18 12:13:09,043 INFO sqlalchemy.engine.Engine [no key 0.00031s] ()
2024-03-18 12:13:09,044 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ADDRESS" 

In [108]:
a = User(name='이름1')
sess.add(a)
sess.commit()

2024-03-18 12:13:16,969 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 12:13:16,974 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2024-03-18 12:13:16,975 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ('이름1',)
2024-03-18 12:13:16,976 INFO sqlalchemy.engine.Engine COMMIT


In [109]:
sess.add(Address(address='주소1', fk=a.pk))
sess.commit()

2024-03-18 12:13:17,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 12:13:17,285 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2024-03-18 12:13:17,288 INFO sqlalchemy.engine.Engine [generated in 0.00265s] (1,)
2024-03-18 12:13:17,292 INFO sqlalchemy.engine.Engine INSERT INTO "T_ADDRESS" ("NAME", "FK") VALUES (?, ?)
2024-03-18 12:13:17,293 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('주소1', 1)
2024-03-18 12:13:17,295 INFO sqlalchemy.engine.Engine COMMIT


In [110]:
sess.query(User.name, Address.address).join(Address).all()

2024-03-18 12:13:17,782 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 12:13:17,787 INFO sqlalchemy.engine.Engine SELECT "T_USER"."NAME" AS "T_USER_NAME", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."NAME" = "T_ADDRESS"."FK"
2024-03-18 12:13:17,789 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ()


[]

In [87]:
                                declarative_base
                   base.metadata.table        base.registiry
                   base.metadata.clear()      base.registriy.dispose()
DB                 ORM;core                    ORM:MAPPER
                   Metadata                    declarative_base
Class(base)        Table객체                    Table 생김
T_USER            Table('T_USER')              User(base)
T_ADDRESS         Table('T_ADDRESS')           Address(base)
                                               Relationship (클래스 사이의 관계를 정의해준다.)
<========================session(engine)========================>
join(T,T,on = Reference/ForeignKey X)

IndentationError: unexpected indent (1632538249.py, line 2)

In [111]:
base.metadata.tables['T_ADDRESS']

Table('T_ADDRESS', MetaData(), Column('PK', Integer(), table=<T_ADDRESS>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_ADDRESS>, nullable=False), Column('FK', Integer(), ForeignKey('T_USER.NAME'), table=<T_ADDRESS>), schema=None)

In [112]:
User.addresses

AttributeError: type object 'User' has no attribute 'addresses'

In [89]:
## 다시 객체만으로

In [141]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.dispose()

engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [142]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import relationship

Base = declarative_base()

class Artist(Base):
    __tablename__ = 'T_ARTIST'
    __table_args__ = {'extend_existing':True}  # 수정 클래스 덮어씌우는

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    albums = relationship("Album", back_populates="artist", uselist=True)

    def __repr__(self):
        return f'pk={self.pk}. name={self.name}'

class Album(Base):
    __tablename__ = 'T_ALBUM'
    __table_args__ = {'extend_existing':True}

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey(Artist.pk))
    artist = relationship("Artist", back_populates="albums", uselist=False)

    def __repr__(self):
        return f'pk={self.pk}. name={self.name}'
        
# class Artist(base):
# class Artist(base):
# class Artist(base):

In [132]:
Artirst.albums[]
1      :       N(userlist=True)
Artist.albums
Album.artist => 경로, backref, back_populates

SyntaxError: invalid syntax (1681694727.py, line 1)

In [143]:
base.metadata.create_all(engine)

2024-03-18 12:31:03,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 12:31:03,564 INFO sqlalchemy.engine.Engine COMMIT


In [144]:
Artist(name='아무개')

pk=None. name=아무개

In [145]:
sess.add(Artist(name='아무개'))

In [ ]:
# 숙제하기 -> POST, HASHTAG